<a href="https://colab.research.google.com/github/Mube2021/MUBE-TECH/blob/main/Copy_of_1st_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install transformers
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
import pandas as pd

In [5]:
# Customize the tokenizer settings as needed
Tokenizer.normalizer = Sequence([NFKC()])
Tokenizer.pre_tokenizer = Whitespace()
Tokenizer.post_processor = None
Tokenizer.decoder = BPE()

In [12]:
# Load the data into a Pandas DataFrame
df = pd.read_csv("datasetv33.csv")


In [13]:

# Initialize a tokenizer with the desired settings
tokenizer = Tokenizer(BPE())

In [14]:
# Train the tokenizer on the text data
trainer = BpeTrainer(special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
tokenizer.train_from_iterator(df["headline"], trainer=trainer)

TypeError: ignored

In [ ]:
# Add special tokens to the tokenizer
tokenizer.add_special_tokens(["[BOS]", "[EOS]"])


In [ ]:
# Encode the text data using the trained tokenizer
encoded_data = []
for text in df["text"]:
    # Add the special tokens to the beginning and end of the text
    text = f"[BOS] {text} [EOS]"
    # Tokenize the text
    tokens = tokenizer.encode(text)
    # Convert the tokens to integer IDs
    ids = tokens.ids
    encoded_data.append(ids)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the integer sequences to the same length
padded_data = pad_sequences(encoded_data, padding="post")


In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and validation sets
train_data, val_data = train_test_split(padded_data, test_size=0.2, random_state=42)


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the pre-trained BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


In [ ]:
import torch

# Convert the data to PyTorch tensors
train_data = torch.tensor(train_data)
val_data = torch.tensor(val_data)


In [ ]:
# Set up the training parameters
num_epochs = 5
batch_size = 32
learning_rate = 1e-5

In [ ]:
# Set up the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()
# Fine-tune the model on the training data
for epoch in range(num_epochs):
    for i in range(0, len(train_data), batch_size):
        # Get the batch of training data
        batch_data = train_data[i:i+batch_size]
        # Clear the gradients
        optimizer.zero_grad()
        # Generate the summary for the batch using the model
        summary_ids = model.generate(batch_data, num_beams=4, max_length=100, early_stopping=True)
        # Calculate the loss between the generated summary and the target summary
        loss = criterion(summary_ids, batch_data)
        # Backpropagate the loss and update the model weights
        loss.backward()
        optimizer.step()

In [ ]:
# Evaluate the model on the validation data after each epoch
    with torch.no_grad():
        val_loss = 0
        for i in range(0, len(val_data), batch_size):
            # Get the batch of validation data
            batch_data = val_data[i:i+batch_size]
            # Generate the summary for the batch using the model
            summary_ids = model.generate(batch_data, num_beams=4, max_length=100, early_stopping=True)
            # Calculate the loss between the generated summary and the target summary
            loss = criterion(summary_ids, batch_data)
            val_loss += loss.item()
        val_loss /= len(val_data) // batch_size
    print(f"Epoch {epoch+1}: Val loss = {val_loss:.4f}")


In [ ]:
#  #this is skipped by gpt but now is here Load the test data
test_data = torch.tensor(padded_test_data)

# Generate summaries for the test data using the fine-tuned model
with torch.no_grad():
    test_summaries =


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
)

trainer.train()


In [ ]:
trainer.evaluate()


In [ ]:
## to generate summary by using fine tuned model use 

In [ ]:
from transformers import pipeline

# load the fine-tuned model
model = AutoModelForSeq2SeqLM.from_pretrained('path/to/fine_tuned_model')

# instantiate the summarization pipeline
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)

# load the test dataset
test_dataset = pd.read_csv('path/to/test_data')

# create a list to store the summaries
summaries = []

# iterate through the test dataset and generate a summary for each news article
for index, row in test_dataset.iterrows():
    article = row['article']
    summary = summarizer(article, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
    summaries.append(summary)

# add the summaries to the test dataset
test_dataset['summary'] = summaries

# save the test dataset with summaries to a CSV file
test_dataset.to_csv('path/to/test_data_with_summaries.csv', index=False)


**To use different models and evaluate them using ROUGE, you can follow these general steps:**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load BERT model
bert_model_name = "bert-base-uncased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSeq2SeqLM.from_pretrained(bert_model_name)


In [ ]:

# Load T5 model
t5_model_name = "t5-base"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Define fine-tuning arguments
bert_training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy = "steps",
    eval_steps = 500,
    save_steps = 500,
    num_train_epochs = 3,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir='./logs',
    logging_steps=1000,
)

# Fine-tune BERT model
bert_trainer = Seq2SeqTrainer(
    model=bert_model,
    args=bert_training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=bert_tokenizer,
)



In [ ]:
t5_training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy = "steps",
    eval_steps = 500,
    save_steps = 500,
    num_train_epochs = 3,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir='./logs',
    logging_steps=1000,
)

# Fine-tune T5 model
t5_trainer = Seq2SeqTrainer(
    model=t5_model,
    args=t5_training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=t5_tokenizer,
)


Evaluate the models: Evaluate each model on your test dataset using the ROUGE metric. You can use the rouge library in Python to compute the ROUGE scores.**bold text**

In [ ]:
from rouge import Rouge
# Initialize the ROUGE metric
rouge = Rouge()
# Evaluate BERT model
bert_predictions = bert_trainer.predict(test_dataset=test_dataset)
bert_predictions = bert_tokenizer.batch_decode(bert_predictions.predictions, skip_special_tokens=True)
bert_scores = rouge.get_scores(bert_predictions, test_targets)



In [ ]:
# Evaluate T5 model
t5_predictions = t5_trainer.predict(test_dataset=test_dataset)
t5_predictions = t5_tokenizer.batch_decode(t5_predictions.predictions, skip_special_tokens=True)
t5_scores = rouge.get_scores(t5_predictions, test_targets)

In [ ]:

# Print the ROUGE scores for both models
print("BERT scores:", bert_scores)
print("T5 scores:", t5_scores)


**to show this values in figurative form usng the above code **

In [ ]:
import matplotlib.pyplot as plt

# Initialize the ROUGE metric
rouge = Rouge()

# Evaluate BERT model
bert_predictions = bert_trainer.predict(test_dataset=test_dataset)
bert_predictions = bert_tokenizer.batch_decode(bert_predictions.predictions, skip_special_tokens=True)
bert_scores = rouge.get_scores(bert_predictions, test_targets)
bert_rouge1_f1 = bert_scores[0]['rouge-1']['f']

# Evaluate T5 model
t5_predictions = t5_trainer.predict(test_dataset=test_dataset)
t5_predictions = t5_tokenizer.batch_decode(t5_predictions.predictions, skip_special_tokens=True)
t5_scores = rouge.get_scores(t5_predictions, test_targets)
t5_rouge1_f1 = t5_scores[0]['rouge-1']['f']

# Plot the ROUGE-1 F1 scores for both models
models = ['BERT', 'T5']
rouge1_f1_scores = [bert_rouge1_f1, t5_rouge1_f1]

plt.bar(models, rouge1_f1_scores)
plt.title('ROUGE-1 F1 Scores')
plt.ylabel('Score')
plt.show()
